In [ ]:
# Pandas for managing datasets
import pandas as pd

# Matplotlib for additional customization
from matplotlib import pyplot as plt
%matplotlib inline
# Seaborn for plotting and styling
import seaborn as sns
# Read dataset
df = pd.read_csv('Pokemon.csv', index_col=0)

spark = SparkSession.builder.master("local[*]").getOrCreate()

sc = spark.sparkContext

textRDD1 = sc.textFile("hobbit.txt")

textRDD2 = spark.read.text('hobbit.txt').rdd

In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

from operator import add
spark = SparkSession \
    .builder.master("local") \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
sc = spark.sparkContext
lines = sc.textFile("C:/AiOLog.txt")
words = lines.flatMap(lambda line: line.split(' '))
wc = words.map(lambda x:(x,1))
counts = wc.reduceByKey(add)
def g(x):
    print (x)
counts.foreach(g)
print(counts.collect())

[('Microsoft', 1), ('.NET', 3), ('Framework', 3), ('4.8', 2), ('(KB4054530)', 1), ('-', 1), ('KB4019990:', 1), ('', 32), ('\tFehlgeschlagen.', 1), ('Fehlercode:', 1), ('2359302', 1), ('(0x00240006)', 1), ('(KB4054530):', 1), ('\tFertig', 1), ('(Reboot', 1), ('required)', 1)]


In [2]:

def mod(x):
    import numpy as np
    return (x, np.mod(x, 2))

rdd = sc.parallelize(range(1000)).map(mod).take(10)
print (rdd)

[(0, 0), (1, 1), (2, 0), (3, 1), (4, 0), (5, 1), (6, 0), (7, 1), (8, 0), (9, 1)]


In [37]:
# spark is an existing SparkSession
from pyspark.sql.types import  (StructType, 
                                StructField, 
                                DateType, 
                                BooleanType,
                                DoubleType,
                                IntegerType,
                                StringType,
                                DecimalType,
                                LongType,
                                ArrayType,
                                TimestampType)
sample_schema = StructType([StructField("id",  IntegerType(), True),
                            StructField("timeAtServer", DoubleType(), True),
                            StructField("aircraft",  IntegerType(), True ),
                            StructField("latitude", DoubleType(), True),
                            StructField("longitude", DoubleType(), True  ),
                            StructField("baroAltitude", DoubleType(), True ),
                            StructField("geoAltitude", DoubleType(), True ),
                            StructField("numM",IntegerType(), True),
                            StructField("measurements", StringType(), True)           
                            ])
df = spark.read.csv("C:/PFE/sample.csv",
                       header = True, 
                        schema = sample_schema)
df.select('aircraft').distinct().count()

439

In [39]:
# Create DataFrame representing the stream of input lines from connection to localhost:9999
fileStreamDf = spark.readStream\
      .option("header", "true")\
      .schema(sample_schema)\
      .csv("C:/PFE/sample.csv")
fileStreamDf.isStreaming

True

In [ ]:
def print_row(row):
      print(row)
print_row=udf(print_row)     
writer = fileStreamDf.writeStream.foreach(print_row)
writer.format("csv")\
      .option("checkpointLocation", "C:/PFE/sampleStream.csv")\
      .outputMode('append').start(path='C:/PFE/sampleStream.csv')

In [ ]:
import pyspark.sql
query = fileStreamDf.writeStream\
      .format("console")\
      .outputMode(Append()).start()
        
query.awaitTermination()

In [5]:
sc

<SparkContext master=local appName=Python Spark SQL basic example>